In [2]:
import os,sys,time
import numpy as np
import cv2

# init camera
camera = cv2.VideoCapture('./testing_videos/2020-12-07-12-12-29_right_camera_pylon_camera_node_image_raw_compressed-10.mp4') ### <<<=== SET THE CORRECT CAMERA NUMBER
#camera.set(3,1280)             # set frame width
#camera.set(4,720)              # set frame height
#time.sleep(0.5)
out = cv2.VideoWriter('2020-12-07-12-12-29_right_camera_pylon_camera_node_image_raw_compressed-12.avi',cv2.VideoWriter_fourcc(*'MJPG'), 10, (600,480))
print(camera.get(3),camera.get(4))

# master frame
master = None
try: 
      
    #creating a folder named data 
    if not os.path.exists('2020-12-07-12-12-29_right_camera_pylon_camera_node_image_raw_compressed-12'): 
        os.makedirs('2020-12-07-12-12-29_right_camera_pylon_camera_node_image_raw_compressed-12') 
  
# if not created then raise error 
except OSError: 
    print ('Error: Creating directory of frame1') 
  
# frame 
currentframe = 0
while 1:

    # grab a frame
    (grabbed,frame0) = camera.read()
    frame0 = cv2.resize(frame0, (600,480),fx = 0, fy = 0,interpolation = cv2.INTER_CUBIC)
    # end of feed

    # gray frame
    frame1 = cv2.cvtColor(frame0,cv2.COLOR_BGR2GRAY)

    # blur frame
    frame2 = cv2.GaussianBlur(frame1,(15,15),0)

    # initialize master
    if master is None:
        master = frame2
        continue

    # delta frame
    frame3 = cv2.absdiff(master,frame2)

    # threshold frame
    frame4 = cv2.threshold(frame3,15,255,cv2.THRESH_BINARY)[1]

    # dilate the thresholded image to fill in holes
    kernel = np.ones((2,2),np.uint8)
    frame5 = cv2.erode(frame4,kernel,iterations=4)
    frame5 = cv2.dilate(frame5,kernel,iterations=8)

    # find contours on thresholded image
    contours,nada = cv2.findContours(frame5.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    # make coutour frame
    frame6 = frame0.copy()

    # target contours
    targets = []

    # loop over the contours
    for c in contours:
        
        # if the contour is too small, ignore it
        if cv2.contourArea(c) < 500:
                continue

        # contour data
        M = cv2.moments(c)#;print( M )
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        x,y,w,h = cv2.boundingRect(c)
        rx = x+int(w/2)
        ry = y+int(h/2)
        ca = cv2.contourArea(c)

        # plot contours
        cv2.drawContours(frame6,[c],0,(0,0,255),2)
        cv2.rectangle(frame6,(x,y),(x+w,y+h),(0,255,0),2)
        cv2.circle(frame6,(cx,cy),2,(0,0,255),2)
        cv2.circle(frame6,(rx,ry),2,(0,255,0),2)

        # save target contours
        targets.append((cx,cy,ca))

    # make target
    mx = 0
    my = 0
    if targets:
        #average centroid adjusted for contour size
        area = 0
        for x,y,a in targets:
            mx += x*a
            my += y*a
            area += a
        mx = int(round(mx/area,0))
        my = int(round(my/area,0))

        # centroid of largest contour
        area = 0
        for x,y,a in targets:
            if a > area:
                mx = x
                my = y
                area = a
    if grabbed: 
        # if video is still left continue creating images 
        name = './2020-12-07-12-12-29_right_camera_pylon_camera_node_image_raw_compressed-12/frame' + str(currentframe) + '.jpg'
        print ('Creating...' + name) 
  
        #writing the extracted images 
        cv2.imwrite(name, frame6) 
  
        #increasing counter so that it will 
        # show how many frames are created 
        currentframe += 1
    else: 
        break   

    # plot target
    tr = 50
    frame7 = frame0.copy()
    if targets:
        #cv2.circle(frame7,(mx,my),tr,(0,0,255,0),2)
        cv2.line(frame7,(mx-tr,my),(mx+tr,my),(0,0,255,0),2)
        cv2.line(frame7,(mx,my-tr),(mx,my+tr),(0,0,255,0),2)
    
    # update master
    master = frame2

    # display
    cv2.imshow("Frame0: Raw",frame0)
    cv2.imshow("Frame3: Delta",frame3)
    cv2.imshow("Frame5: Dialated",frame5)
    cv2.imshow("Frame6: Contours",frame6)
    cv2.imshow("Frame7: Target",frame7)
    #out.write(frame6)
    # key delay and action
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key != 255:
        print('key:',[chr(key)])

# release camera
camera.release()
#out.release()
# close all windows
cv2.destroyAllWindows()

SyntaxError: invalid syntax (<ipython-input-2-51cb369b4c07>, line 94)